In [35]:
import time
import requests as req
import re
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd

In [36]:
location_webdriver = r"C:\Users\Manal\Downloads\chromedriver_win32\chromedriver.exe"
driver = webdriver.Chrome(location_webdriver)

<ipython-input-36-412fcf5427c8>:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(location_webdriver)


In [37]:
def get_page_info(soup, channel_title):
                        
    CHANNEL_TYPE = []

    for i in (soup.find_all('div', {'id': 'YouTubeUserTopInfoBlockTop'})) :
        rows = i.find_all('div', {'class': 'YouTubeUserTopInfo' })
        for j in rows :
            cols = j.find_all('a', {'id' : 'youtube-user-page-channeltype', 'style' : "color:#555;"})
            for index, row in enumerate(cols) :
                if index == 0 :
                    CHANNEL_TYPE.append(row.text.strip())
                else :
                    continue
    
    
    headers = ['Channel', 'CHANNEL_TYPE']
    
    return dict(zip(headers, [channel_title,
                                    CHANNEL_TYPE,
                                    ]))
    


In [38]:
driver.get("https://socialblade.com/youtube/")
search = driver.find_element_by_xpath('//*[@id="SearchInput"]')
search.send_keys('T-Series')

<ipython-input-38-17672ea67177>:2: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  search = driver.find_element_by_xpath('//*[@id="SearchInput"]')


In [39]:
driver.find_element_by_xpath('/html/body/div[7]/div/div[2]/form/div[1]/button/i').click()
time.sleep(5)
driver.get(driver.current_url)
soup = BeautifulSoup(driver.page_source)

<ipython-input-39-7ef0726e10a2>:1: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  driver.find_element_by_xpath('/html/body/div[7]/div/div[2]/form/div[1]/button/i').click()


In [40]:
pd.DataFrame(get_page_info(soup, 'T-Series'))

,Channel,CHANNEL_TYPE
0,T-Series,Music


In [41]:
df = pd.read_csv("Top_1000_Youtube_Channels.csv")
df.head()

,Pos,Channel,Views,Subscriber,Vidoes,Published
0,1,T-Series,"157,452,055,214","185,000,000","15,528",2006-03-13
1,2,Cocomelon - Nursery Rhymes,"103,803,532,922","113,000,000",654,2006-09-01
2,3,SET India,"90,768,285,232","107,000,000","59,421",2006-09-20
3,4,WWE,"59,710,652,499","78,500,000","55,466",2007-05-11
4,5,Like Nastya,"58,866,363,302","74,700,000",589,2016-12-06


In [42]:
df_list = []
counter = 0

for channel in df.Channel.values:
    search = driver.find_element_by_xpath('//*[@id="SearchInput"]')
    search.send_keys(channel)
    driver.find_element_by_xpath('/html/body/div[7]/div/div[2]/form/div[1]/button/i').click()
    time.sleep(10)
    driver.get(driver.current_url)
    soup = BeautifulSoup(driver.page_source)
    df_list.append(pd.DataFrame(get_page_info(soup, channel)) )

<ipython-input-42-21b566c89813>:5: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  search = driver.find_element_by_xpath('//*[@id="SearchInput"]')
<ipython-input-42-21b566c89813>:7: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  driver.find_element_by_xpath('/html/body/div[7]/div/div[2]/form/div[1]/button/i').click()


In [43]:
df_new = pd.concat(df_list)
df_new

,Channel,CHANNEL_TYPE
0,T-Series,Music
0,Cocomelon - Nursery Rhymes,Education
0,SET India,Entertainment
0,WWE,Sports
0,Like Nastya,Entertainment
...,...,...
0,또봇 V · 메탈리온 [Tobot V · Metalions],Film
0,planbVEVO,Music
0,TheSoundYouNeed,Music
0,Ch3Thailand Music,Music


In [45]:
df_new.to_csv("SocialBlade_type.csv", index = False)